# サーチエンジンとしての利用手順(運用)


## 準備

本章のコマンドを実行するための設定を行います。  

セットアップ済みの環境のうちコマンドを送る先であるCoordinating(Client) Nodeのホストアドレスを、次のセルに記入して実行し、保存してください。

In [53]:
%env ES_HOST=XXX.XXX.XXX.233:9200

env: ES_HOST=XXX.XXX.XXX.233:9200


また、データを蓄積する先のインデックスとタイプを、次のセルを実行して保存してください。  

※特にセルの内容の書き換えは必要ありません。  
　前の章と異なるインデックス名にしているのは、この章のコマンドで投入する内容が前の章までのものと混ざらないようにするためです。

In [47]:
%env INDEX=meteorological-data-*
%env TYPE=logs

env: INDEX=meteorological-data-*
env: TYPE=log


## ノード追加・障害復旧時の注意点

### 発生する可能性のある問題点

1. データが入った状態のノードを追加した場合、ドキュメントIDが重複してしまう可能性があります。  
    その場合追加された方が無視されるので結果としてデータが入っていないように見えてしまいます。  
    ノードを追加する場合はデータが入っていないものを追加するようにしてください。
1. ノードが追加、または削除された場合、クラスタのデータが再構成されて均等に分散されます。  
    この処理にはCPU,メモリを消費するためその期間は性能が落ちます。Task Management APIを使用して実行中のイベントを確認することができます。  
    Task APIについてはElasticsearch Referenceの[Task Management API](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/tasks.html)を参照してください。

ノード内の実行中のイベントを確認するコマンドは次の通りです。

In [48]:
%%bash
curl -XGET "http://$ES_HOST/_tasks?group_by=parents&pretty"

{
  "tasks" : {
    "nuc7d3LiTFamTIsEQgo--w:1907" : {
      "node" : "nuc7d3LiTFamTIsEQgo--w",
      "id" : 1907,
      "type" : "transport",
      "action" : "cluster:monitor/tasks/lists",
      "start_time_in_millis" : 1481515946937,
      "running_time_in_nanos" : 581465,
      "cancellable" : false,
      "children" : [
        {
          "node" : "mBJcG4SZRF6BiVftkttXkQ",
          "id" : 2043,
          "type" : "netty",
          "action" : "cluster:monitor/tasks/lists[n]",
          "start_time_in_millis" : 1481515946938,
          "running_time_in_nanos" : 110411,
          "cancellable" : false,
          "parent_task_id" : "nuc7d3LiTFamTIsEQgo--w:1907"
        },
        {
          "node" : "JjtDV9M6SBeUQtcAfgZaBQ",
          "id" : 7051,
          "type" : "netty",
          "action" : "cluster:monitor/tasks/lists[n]",
          "start_time_in_millis" : 1481515946938,
          "running_time_in_nanos" : 188913,
          "cancellable" : false,
          "parent_task_id" :

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2136  100  2136    0     0   251k      0 --:--:-- --:--:-- --:--:--  260k


### Cluster APIsを利用して状況を把握する方法

ノード追加・削除等変更を行った際に、クラスタの状態を確認する方法は次の通りです。

In [49]:
%%bash
curl -XGET "http://$ES_HOST/_cluster/health?pretty"

{
  "cluster_name" : "cluster-name",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 5,
  "number_of_data_nodes" : 3,
  "active_primary_shards" : 65,
  "active_shards" : 130,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   468  100   468    0     0  60488      0 --:--:-- --:--:-- --:--:-- 66857


クラスタの状態はred,yellow,greenの3種類あります。  


| 状態 | 意味 |
|-----|-----|
| green | すべてのシャードが割り当てられている |
| yellow | プライマリシャードが割り当てられているがレプリカは割り当てられていない |
| red | クラスタ内に特定のシャードが割り当てられていない |

yellowでも動作しますが、障害発生時にデータがなくなる可能性が高まります。  
平常時は常にgreenにしておくことを推奨します。  
red,yellowの場合ノード停止などの障害が発生しているため、復旧して状態をgreenにしてください。

## force merge APIを実行する必要性

ElasticsearchのDELETE APIを使用したレコードの削除処理は、削除対象レコードに対して削除フラグを設定し、  
検索対象から除外されるだけとなります。  
そのため実データは残留し、リソースを使用したままになります。

force merge APIは、削除フラグが設定されたレコードの実データを削除し、インデックスの最適化を行いシャードのセグメント数を削減します。  
これにより、リソース使用量を削減することができます。  
force merge APIについてはElasticsearch Referenceの[Force Merge](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/indices-forcemerge.html)を参照してください。  

force mergeは以下の場合に必要となります。  
* データを完全に削除し、ディスク領域を空ける
* ドキュメントIDを解放する

削除フラグがonの状態のデータが何件あるかはIndices Statsで確認することができます。
Elasticsearch Referenceの[Indices Stats](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/indices-stats.html)を参照してください。

* 削除フラグ件数の確認方法は次の通りです。

In [50]:
%%bash
curl -XGET "http://$ES_HOST/$INDEX/_stats?pretty"

{
  "_shards" : {
    "total" : 130,
    "successful" : 130,
    "failed" : 0
  },
  "_all" : {
    "primaries" : {
      "docs" : {
        "count" : 359,
        "deleted" : 0
      },
      "store" : {
        "size_in_bytes" : 782733,
        "throttle_time_in_millis" : 0
      },
      "indexing" : {
        "index_total" : 0,
        "index_time_in_millis" : 0,
        "index_current" : 0,
        "index_failed" : 0,
        "delete_total" : 0,
        "delete_time_in_millis" : 0,
        "delete_current" : 0,
        "noop_update_total" : 0,
        "is_throttled" : false,
        "throttle_time_in_millis" : 0
      },
      "get" : {
        "total" : 0,
        "time_in_millis" : 0,
        "exists_total" : 0,
        "exists_time_in_millis" : 0,
        "missing_total" : 0,
        "missing_time_in_millis" : 0,
        "current" : 0
      },
      "search" : {
        "open_contexts" : 0,
        "query_total" : 33,
        "query_time_in_millis" : 96,
        "query_current"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 94830  100 94830    0     0  2066k      0 --:--:-- --:--:-- --:--:-- 2104k


"deleted"に表示されている件数が削除フラグが設定されているドキュメントの数になります。

* force merge APIの実行方法は次の通りです。

In [52]:
%%bash
curl -XPOST "http://$ES_HOST/$INDEX/_forcemerge?pretty"

{
  "_shards" : {
    "total" : 130,
    "successful" : 130,
    "failed" : 0
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    84  100    84    0     0   5309      0 --:--:-- --:--:-- --:--:--  5600


## データバックアップ

Elasticsearchのバックアップはクラスタ・index単位で取得することができます。  
詳細はElasticsearch Referenceの[Snapshot And Restore](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-snapshots.html)を参照してください。

### バックアップ対象indexの容量確認

バックアップの取得前にindexの容量を確認し、十分な空きが保存ストレージにあることを確認してください。  
indexの容量はcat APIにより確認できます。  
詳細はElasticsearch Referenceの[cat-indices](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/cat-indices.html)を参照してください。

In [80]:
%%bash
curl -XGET "http://$ES_HOST/_cat/indices?v"

health status index          uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   sample_index   NCA-vz62SfisSEoiBBdyHA   5   1        110            0    907.1kb        453.5kb
green  open   reindex_source FTQ0QcHxR9qVpt_sART-YQ   5   1          1            0      8.9kb          4.4kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   330  100   330    0     0  24422      0 --:--:-- --:--:-- --:--:-- 25384


### クラスタ単位のバックアップ

システム全体のデータバックアップを取得する場合に実行してください。  

例として、取得手順は次の通りです。

* バックアップを保存するリポジトリ(my_backup)を作成します

In [56]:
%%bash
curl -XPUT "http://$ES_HOST/_snapshot/my_backup" -d @- << EOF
{
    "type": "fs",
    "settings": {
        "location": "/mnt/backup/my_backup",
        "compress": true
    }
}
EOF

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   130  100    21  100   109    229   1193 --:--:-- --:--:-- --:--:--  1197


* my_backupリポジトリが作成されたことを確認します

In [57]:
%%bash
curl -XGET "http://$ES_HOST/_snapshot/my_backup?pretty"

{
  "my_backup" : {
    "type" : "fs",
    "settings" : {
      "compress" : "true",
      "location" : "/mnt/backup/my_backup"
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   140  100   140    0     0   8829      0 --:--:-- --:--:-- --:--:--  9333


* my_backupリポジトリにsnapshot_1という名前でバックアップを取得します

In [58]:
%%bash
curl -XPUT "http://$ES_HOST/_snapshot/my_backup/snapshot_1?wait_for_completion=true&pretty"

{
  "snapshot" : {
    "snapshot" : "snapshot_1",
    "uuid" : "3T5V4TwsQ7C1a_O0cqO7NA",
    "version_id" : 5000099,
    "version" : "5.0.0",
    "indices" : [
      "sample_index",
      "meteorological-data-2015.07",
      "meteorological-data-2015.08",
      "meteorological-data-2015.03",
      "meteorological-data-2015.09",
      "meteorological-data-2015.05",
      "meteorological-data-2015.01",
      "meteorological-data-2015.04",
      "meteorological-data-2015.12",
      "meteorological-data-2015.11",
      "meteorological-data-2015.10",
      "meteorological-data-2015.02",
      "meteorological-data-2015.06",
      "meteorological-data-2014.12"
    ],
    "state" : "SUCCESS",
    "start_time" : "2016-12-13T04:31:41.470Z",
    "start_time_in_millis" : 1481603501470,
    "end_time" : "2016-12-13T04:31:42.079Z",
    "end_time_in_millis" : 1481603502079,
    "duration_in_millis" : 609,
    "failures" : [ ],
    "shards" : {
      "total" : 70,
      "failed" : 0,
      "successful

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1019  100  1019    0     0   1555      0 --:--:-- --:--:-- --:--:--  1555


* バックアップ(snapshot_1)が取得できたことを確認します

In [59]:
%%bash
curl -XGET "http://$ES_HOST/_snapshot/my_backup/snapshot_1?pretty"

{
  "snapshots" : [
    {
      "snapshot" : "snapshot_1",
      "uuid" : "3T5V4TwsQ7C1a_O0cqO7NA",
      "version_id" : 5000099,
      "version" : "5.0.0",
      "indices" : [
        "sample_index",
        "meteorological-data-2015.07",
        "meteorological-data-2015.08",
        "meteorological-data-2015.03",
        "meteorological-data-2015.09",
        "meteorological-data-2015.05",
        "meteorological-data-2015.01",
        "meteorological-data-2015.04",
        "meteorological-data-2015.12",
        "meteorological-data-2015.11",
        "meteorological-data-2015.10",
        "meteorological-data-2015.02",
        "meteorological-data-2015.06",
        "meteorological-data-2014.12"
      ],
      "state" : "SUCCESS",
      "start_time" : "2016-12-13T04:31:41.470Z",
      "start_time_in_millis" : 1481603501470,
      "end_time" : "2016-12-13T04:31:42.079Z",
      "end_time_in_millis" : 1481603502079,
      "duration_in_millis" : 609,
      "failures" : [ ],
      "shards

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1096  100  1096    0     0  72823      0 --:--:-- --:--:-- --:--:-- 78285


### index単位のバックアップ

indexを指定してバックアップを取得したい場合に実行してください。  
例えば、meteorological-data-\* indexを指定してバックアップを取得する手順は次の通りです。

* バックアップを保存するリポジトリ(my_backup)を作成します

In [63]:
%%bash
curl -XPUT "http://$ES_HOST/_snapshot/my_backup" -d @- << EOF
{
    "type": "fs",
    "settings": {
        "location": "/mnt/backup/my_backup",
        "compress": true
    }
}
EOF

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   130  100    21  100   109    372   1931 --:--:-- --:--:-- --:--:--  1946


* my_backupリポジトリが作成されたことを確認します

In [70]:
%%bash
curl -XGET "http://$ES_HOST/_snapshot/my_backup?pretty"

{
  "my_backup" : {
    "type" : "fs",
    "settings" : {
      "compress" : "true",
      "location" : "/mnt/backup/my_backup"
    }
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   140  100   140    0     0  34465      0 --:--:-- --:--:-- --:--:-- 46666


* my_backupリポジトリにindexを指定して、snapshot_1という名前でバックアップ取得します  
オプションについては、Elasticsearch Referenceの[Snapshot And Restore](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-snapshots.html)を参照してください。

In [71]:
%%bash
curl -XPUT "http://$ES_HOST/_snapshot/my_backup/snapshot_1" -d @- << EOF
{
  "indices": "$INDEX",
  "ignore_unavailable": true,
  "include_global_state": false
}
EOF

{"accepted":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   116  100    17  100    99    153    891 --:--:-- --:--:-- --:--:--   900


* バックアップ(snapshot_1)が取得できたことを確認します

In [73]:
%%bash
curl -XGET "http://$ES_HOST/_snapshot/my_backup/snapshot_1?pretty"

{
  "snapshots" : [
    {
      "snapshot" : "snapshot_1",
      "uuid" : "vSt3NquaSX-z6CYuWER9jw",
      "version_id" : 5000099,
      "version" : "5.0.0",
      "indices" : [
        "meteorological-data-2015.07",
        "meteorological-data-2015.08",
        "meteorological-data-2015.03",
        "meteorological-data-2015.09",
        "meteorological-data-2015.05",
        "meteorological-data-2015.01",
        "meteorological-data-2015.04",
        "meteorological-data-2015.12",
        "meteorological-data-2015.10",
        "meteorological-data-2015.11",
        "meteorological-data-2015.06",
        "meteorological-data-2015.02",
        "meteorological-data-2014.12"
      ],
      "state" : "SUCCESS",
      "start_time" : "2016-12-13T04:52:28.539Z",
      "start_time_in_millis" : 1481604748539,
      "end_time" : "2016-12-13T04:52:28.958Z",
      "end_time_in_millis" : 1481604748958,
      "duration_in_millis" : 419,
      "failures" : [ ],
      "shards" : {
        "total" : 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1072  100  1072    0     0   122k      0 --:--:-- --:--:-- --:--:--  130k


## データリストア

### データリストア前確認

データリストアはindexが空のところに入れることを前提としています。  
リストア実施前に、同名indexが存在しないことを確認してください。  

現状存在するindexを確認するコマンドは、次の通りです。  
詳細はElasticsearch Referenceの[cat-indices](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/cat-indices.html)を参照してください。

In [81]:
%%bash
curl -XGET "http://$ES_HOST/_cat/indices?v"

health status index          uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   reindex_dest   hm0itWDWTP-h05dMnJSRAQ   5   1          1            0      8.8kb          4.4kb
green  open   reindex_source 0CVwNJ0XQievecjHPq5kNQ   5   1          1            0      8.9kb          4.4kb
green  open   sample_index   NCA-vz62SfisSEoiBBdyHA   5   1        110            0    907.1kb        453.5kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   440  100   440    0     0  28299      0 --:--:-- --:--:-- --:--:-- 29333


また、snapshotから一部のindexをリストアすることやindex名を変更しながらリストアすることも可能ですので、  
状況に応じて必要なindexをリストアしてください。    
詳細はElasticsearch Referenceの[Restoring from a Snapshotedit](https://www.elastic.co/guide/en/elasticsearch/reference/5.0/modules-snapshots.html)を参照してください。

### データリストア実行

取得したバックアップファイルからデータをリストアすることができます。  
例として、1.4で取得したバックアップ(snapshot_1)からリストアするコマンドは次の通りです。

In [77]:
%%bash
curl -XPOST "http://$ES_HOST/_snapshot/my_backup/snapshot_1/_restore?pretty"

{
  "accepted" : true
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    24  100    24    0     0    239      0 --:--:-- --:--:-- --:--:--   242


## データの削除方法
データの削除方法には以下3パターンあります。
1. index単位で削除
1. ドキュメントIDを指定して特定のデータを削除
1. 条件を指定して削除

ドキュメントIDを指定、条件を指定して削除する方法については[02_Search](02_Search.ipynb)を参照してください。

### index単位で削除

indexが「meteorological-data-\*」の場合、削除コマンドは次の通りです。

In [79]:
%%bash
curl -XDELETE "http://$ES_HOST/meteorological-data-*/"

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    21  100    21    0     0     98      0 --:--:-- --:--:-- --:--:--    98


** 注意 ** 次のコマンドを実行した場合データが全て削除され、復旧することができなくなるため注意してください。

DELETE APIについては、Elasticsearch Referenceの[Delete API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-delete.html)を参照してください。